In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split

In [11]:
from scraping import scraper
import numpy as np
import pandas as pd

In [15]:
years = np.arange(2000, 2023)
scraper = scraper(years = years)
games = scraper.fit()

... 2021-2022 ...  (23/23)


In [16]:
games = pd.concat([games, pd.get_dummies(games['Result'].replace(1,'Win').replace(0,'Draw').replace(-1,'Loss'))], axis = 1)

In [17]:
games

,Date,Season,Game_home,Game_away,Home,Away,Result,G_home,G_away,PTS%,...,GA_inf,GD_1,GD_2,GD_5,GD_10,GD_inf,Rest,Draw,Loss,Win
0,1999-08-07,2000,1,1,Arsenal,Leicester City,1,2,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1
1,1999-08-07,2000,1,1,Newcastle Utd,Aston Villa,-1,0,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0
2,1999-08-07,2000,1,1,Middlesbrough,Bradford City,-1,0,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0
3,1999-08-07,2000,1,1,Chelsea,Sunderland,1,4,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1
4,1999-08-07,2000,1,1,Sheffield Weds,Liverpool,-1,1,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8735,2022-05-22,2022,38,38,Brighton,West Ham,1,3,1,-0.072072,...,-0.135135,0.0,0.0,0.4,0.3,-0.405405,0.0,0,0,1
8736,2022-05-22,2022,38,38,Brentford,Leeds United,-1,1,2,0.099099,...,-0.648649,1.0,3.5,2.0,1.6,0.837838,0.0,0,1,0
8737,2022-05-22,2022,38,38,Arsenal,Everton,1,5,1,0.243243,...,-0.378378,-3.0,-2.5,-0.6,-0.2,0.756757,3.0,0,0,1
8738,2022-05-22,2022,38,38,Crystal Palace,Manchester Utd,1,1,0,-0.117117,...,-0.270270,3.0,0.0,1.6,1.3,0.054054,-12.0,0,0,1


In [50]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense

# Load historical match data
data = games.copy()[(games['Game_home'] >= 5) & (games['Game_away'] >= 5)]
#train, test = train_test_split(data, test_size = 0.1)
train = data[data['Season'] <  2022]
test  = data[data['Season'] == 2022]
X_train, X_test = np.array(train[scraper.features]), np.array(test[scraper.features])
y_train, y_test = np.array(train[['Win', 'Draw', 'Loss']]), np.array(test[['Win', 'Draw', 'Loss']])

# Create neural network model
model = Sequential()
model.add(Dense(32, activation = 'relu', input_dim = len(scraper.features)))
model.add(Dense(16, activation = 'relu'))
model.add(Dense(8,  activation = 'relu'))
model.add(Dense(3,  activation = 'softmax'))

# Compile model
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

# Train model
model.fit(X_train, y_train, epochs = 50, batch_size = 32)

Epoch 1/50
233/233 [==============================] - 1s 527us/step - loss: 1.0610 - accuracy: 0.4532
Epoch 2/50
233/233 [==============================] - 0s 500us/step - loss: 1.0104 - accuracy: 0.5142
Epoch 3/50
233/233 [==============================] - 0s 516us/step - loss: 0.9989 - accuracy: 0.5170
Epoch 4/50
233/233 [==============================] - 0s 531us/step - loss: 0.9945 - accuracy: 0.5187
Epoch 5/50
233/233 [==============================] - 0s 500us/step - loss: 0.9922 - accuracy: 0.5201
Epoch 6/50
233/233 [==============================] - 0s 521us/step - loss: 0.9900 - accuracy: 0.5227
Epoch 7/50
233/233 [==============================] - 0s 552us/step - loss: 0.9896 - accuracy: 0.5244
Epoch 8/50
233/233 [==============================] - 0s 539us/step - loss: 0.9880 - accuracy: 0.5226
Epoch 9/50
233/233 [==============================] - 0s 541us/step - loss: 0.9872 - accuracy: 0.5246
Epoch 10/50
233/233 [==============================] - 0s 492us/step - loss: 0.985

In [51]:
pred = model.predict(X_test)
test[['W_pred', 'D_pred', 'L_pred']] = pred

11/11 [==============================] - 0s 531us/step


/var/folders/x8/gymrgfns32bd6rd_ksf19p3m0000gn/T/ipykernel_33837/1302692932.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[['W_pred', 'D_pred', 'L_pred']] = pred
/var/folders/x8/gymrgfns32bd6rd_ksf19p3m0000gn/T/ipykernel_33837/1302692932.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[['W_pred', 'D_pred', 'L_pred']] = pred
/var/folders/x8/gymrgfns32bd6rd_ksf19p3m0000gn/T/ipykernel_33837/1302692932.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

In [52]:
p = np.zeros_like(pred)
p[np.arange(len(pred)), pred.argmax(1)] = 1
test['RW'] = 1 - np.sign(np.abs(y_test - p).sum(axis = 1))

/var/folders/x8/gymrgfns32bd6rd_ksf19p3m0000gn/T/ipykernel_33837/1268772021.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['RW'] = 1 - np.sign(np.abs(y_test - p).sum(axis = 1))
